In [170]:
#Importing pandas package which is the main package
import pandas as pd
#Importing the supporting numpy package to perform mathematical computations if any
import numpy as np
#Importing the packages that support visualization
import matplotlib.pyplot as plt

#### This notebook is the final version after doing a lot of small checks

In [171]:
#Step 1 : Import/Read the CSV file provided
data = pd.read_csv('loan.csv')

C:\Users\Suraj MU\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [172]:
#Looking at the Sample Rows
data.head()

id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  1077501    1296599       5000         5000           4975.0   36 months   
1  1077430    1314167       2500         2500           2500.0   60 months   
2  1077175    1313524       2400         2400           2400.0   36 months   
3  1076863    1277178      10000        10000          10000.0   36 months   
4  1075358    1311748       3000         3000           3000.0   60 months   

  int_rate  installment grade sub_grade            ...              \
0   10.65%       162.87     B        B2            ...               
1   15.27%        59.83     C        C4            ...               
2   15.96%        84.33     C        C5            ...               
3   13.49%       339.31     C        C1            ...               
4   12.69%        67.79     B        B5            ...               

  num_tl_90g_dpd_24m num_tl_op_past_12m pct_tl_nvr_dlq  percent_bc_gt_75  \
0                NaN                NaN            NaN               NaN   
1                NaN                NaN            NaN               NaN   
2                NaN                NaN            NaN               NaN   
3                NaN                NaN            NaN               NaN   
4                NaN                NaN            NaN               NaN   

  pub_rec_bankruptcies tax_liens tot_hi_cred_lim total_bal_ex_mort  \
0                  0.0       0.0             NaN               NaN   
1                  0.0       0.0             NaN               NaN   
2                  0.0       0.0             NaN               NaN   
3                  0.0       0.0             NaN               NaN   
4                  0.0       0.0             NaN               NaN   

  total_bc_limit total_il_high_credit_limit  
0            NaN                        NaN  
1            NaN                        NaN  
2            NaN                        NaN  
3            NaN                        NaN  
4            NaN                        NaN  

[5 rows x 111 columns]

In [173]:
#Get to know how many rows and columns are there
data.shape

(39717, 111)

#### The y-variable is loan_status and is the main variable that needs to be analysed with other x variables

In [174]:
data['loan_status'].value_counts()

Fully Paid     32950
Charged Off     5627
Current         1140
Name: loan_status, dtype: int64

### We observe there are lot of columns in this dataset, best approach would be to go through the data dictionary and identify logical/ business/ data quality related reasons to eliminate redundant columns before starting the analysis, so as to arrive at a Analytical Dataset to perform the analysis

In [175]:
#Checking Null Value Counts in each column
data.isnull().sum()

id                                    0
member_id                             0
loan_amnt                             0
funded_amnt                           0
funded_amnt_inv                       0
term                                  0
int_rate                              0
installment                           0
grade                                 0
sub_grade                             0
emp_title                          2459
emp_length                         1075
home_ownership                        0
annual_inc                            0
verification_status                   0
issue_d                               0
loan_status                           0
pymnt_plan                            0
url                                   0
desc                              12940
purpose                               0
title                                11
zip_code                              0
addr_state                            0
dti                                   0


### 1. We observe that a lot of columns have null values entirely, hence we can eliminate those columns (We can define a % of missing values logic as well, but it is straightforward in this dataset

In [176]:
#Visited Pandas dropna documentation to understand its parameters with examples : https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html
#Condition specified below is to drop columns which have NAs/Nulls Completely
data1 = data.dropna(how='all',axis=1)

In [177]:
#Re checking the shape of the dataframe again, we were able eliminate close to 54 columns with the above condition
data1.shape

(39717, 57)

In [178]:
#Checking the sum of NA/NULLs again
data1.isnull().sum()

id                                0
member_id                         0
loan_amnt                         0
funded_amnt                       0
funded_amnt_inv                   0
term                              0
int_rate                          0
installment                       0
grade                             0
sub_grade                         0
emp_title                      2459
emp_length                     1075
home_ownership                    0
annual_inc                        0
verification_status               0
issue_d                           0
loan_status                       0
pymnt_plan                        0
url                               0
desc                          12940
purpose                           0
title                            11
zip_code                          0
addr_state                        0
dti                               0
delinq_2yrs                       0
earliest_cr_line                  0
inq_last_6mths              

In [179]:
#We again observe that there are few columns with NA/NULL values close to the length of the column, removing them manually
data2 = data1.drop(['mths_since_last_delinq','mths_since_last_record','next_pymnt_d'], axis = 1)

In [180]:
#Checking Shape of dataframe again (Removed 3 columns manually)
data2.shape

(39717, 54)

### 2. After removing columns which have NULL values entirely, it is now time to remove unecessary columns that does not fit our use case
#### 1. {id, member_id} - These are random numbers used to identify the individual, will not help in the analysis
#### 2. {issue_d, initial_list_status, out_prncp, out_prncp_inv, total_rec_prncp, total_rec_int, total_rec_late_fee, recoveries, collection_recovery_fee,  last_pymnt_d, last_pymnt_amnt, last_credit_pull_d, collections_12_mths_ex_med, funded_amt, funded_amt_inv, installment, total_pymnt, total_pymnt_inv, revol_bal, revol_util} - These paramters are absent during the application
#### 3. {url} - Its the url for the LC Page listing data, will not help in the analysis
#### 4. {title, desc} - Can use the purpose column, easily categorized, its redundant
#### 4. {emp_title} - This is a descriptive column, with a lot of messy values(For ex: US Army, U.S. Army, US ARMY) in terms of additional special characters,Short Forms and long forms, different case for the same string, its also mentioned that the emp_title shall replace names post Sep 23, 2013, going forward in the future also this column won't be useful, hence deleting this column, because even if it makes sense to the analysis and if some insights do come out of it, this wont be captured going forward

In [181]:
#Removing additional columns as they wont be available during the loan application process and wont help us understand about a loan defaulter
data3 = data2.drop(['id','member_id','issue_d','url','desc','title','initial_list_status','out_prncp','out_prncp_inv',
                   'total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee',
                   'last_pymnt_d','last_pymnt_amnt','last_credit_pull_d','collections_12_mths_ex_med','funded_amnt',
                   'funded_amnt_inv','installment','total_pymnt','total_pymnt_inv','emp_title','revol_bal','revol_util',
                   'pub_rec','pub_rec_bankruptcies'], axis = 1)

In [182]:
#Rechecking the Shape
data3.shape

(39717, 26)

### 3. Removing columns that are single valued as they don't add value to our analysis

In [183]:
#Checking all the columns which are single valued columns
data3.columns[data3.nunique() <= 1]

Index(['pymnt_plan', 'policy_code', 'application_type', 'acc_now_delinq',
       'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens'],
      dtype='object')

In [184]:
#Dropping the above columns
data4 = data3.drop(['pymnt_plan', 'policy_code', 'application_type', 'acc_now_delinq',
       'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens'], axis=1)

In [185]:
#Checking Shape again
data4.shape

(39717, 19)

### 4. Our goal is to identify factors influencing a loan default and hence we are not interested in the records where the individuals are still paying currently 

In [186]:
#Checking how many rows of each of the loan statuses are present
data4['loan_status'].value_counts()

Fully Paid     32950
Charged Off     5627
Current         1140
Name: loan_status, dtype: int64

#### <i> If we look at the result above, we see that majority of records are of the 'Fully Paid' status, that of the Charged Off status has less values already, in our use case Charged Off status is the important status and we are looking for factors influencing them, so we should take care so that we dont remove rows of the Charged Off status </i>

In [187]:
#Retaining rows which has a Full Paid status or Charged Off status
data5 = data4[(data4['loan_status']=='Fully Paid') | (data4['loan_status']=='Charged Off')]

In [188]:
#Rechecking the shape
data5.shape

(38577, 19)

In [189]:
#Checking the sum of NA/NULLs again
data5.isnull().sum()

loan_amnt                 0
term                      0
int_rate                  0
grade                     0
sub_grade                 0
emp_length             1033
home_ownership            0
annual_inc                0
verification_status       0
loan_status               0
purpose                   0
zip_code                  0
addr_state                0
dti                       0
delinq_2yrs               0
earliest_cr_line          0
inq_last_6mths            0
open_acc                  0
total_acc                 0
dtype: int64

#### We observe that in the latest altered dataset, missing values are present in emp_length column

In [191]:
#Checking how many values in each of the loan status
data5['loan_status'].value_counts()

Fully Paid     32950
Charged Off     5627
Name: loan_status, dtype: int64

##### There are 1033 missing rows in emp_length column, trying to remove this

In [192]:
data6 = data5[~data5['emp_length'].isnull()]

In [195]:
data6.shape

(37544, 19)

In [193]:
#Checking how many values in each of the loan status
data6['loan_status'].value_counts()

Fully Paid     32145
Charged Off     5399
Name: loan_status, dtype: int64

##### After removing NULLs in the emp_length column, didnt end up removing a lot of rows of the Charged off Segment

In [194]:
#Rechecking the NULLs
data6.isnull().sum()

loan_amnt              0
term                   0
int_rate               0
grade                  0
sub_grade              0
emp_length             0
home_ownership         0
annual_inc             0
verification_status    0
loan_status            0
purpose                0
zip_code               0
addr_state             0
dti                    0
delinq_2yrs            0
earliest_cr_line       0
inq_last_6mths         0
open_acc               0
total_acc              0
dtype: int64

#### Have removed columns and rows that does not fit the context of the problem